This is version 2 of my Fantasy Hockey Analyzer. The purpose of this notebook is to predict the number of fantasy points every hockey player in the league will get based on previous years' performance.

This notebook primarily uses data from moneypuck.com for analysis, and it also uses data from rotowire.com to get +/- for each player.

Section 1: Parameters and Modules

These are the variables that can be adjusted. My model is an ensemble model consisting of neural nets and random forests, with data going back one, two, and three years.

In [1]:
# Set these values to the appropriate ammonts

current_year = 2025
common_number = 1
number_of_one_year_neural_nets = common_number
number_of_two_year_neural_nets = common_number
number_of_three_year_neural_nets = common_number
number_of_one_year_random_forests = common_number
number_of_two_year_random_forests = common_number
number_of_three_year_random_forests = common_number

# This is the breakdown of how many fantasy points a player gets for each category
points_dictionary = {
    'Goals':5, 
    'Assists':3, 
    '+/-':1.5, 
    'PIM':-0.25, 
    'PP_Goals':4, 
    'PP_Assists':2, 
    'SH_Goals':6, #won't count SHG from 5-on-3
    'SH_Assists':4, 
    'Faceoffs_Won':0.25, 
    'Faceoffs_Lost':-0.15, 
    'Hits':0.5, 
    'Blocked_Shots':0.75
    }

The following is a list of modules that I used and the reason why they were used:

-os: to allow the program to read data in the repository

-numpy: basic math operations

-pandas: all dataframe operations/data storage/data cleaning

-various sklearn: all machine learning operations/analysis

In addition to these modules, I also have a custom module that contains helper functions that help in data cleaning/accuracy evaluation. These functions are contained in the "my_module.py" file in the repository. If you are interested in taking a look at these functions, they are available at https://github.com/chrisberry888/FantasyHockeyAnalyzer in the "my_module.py" file.

In [2]:
#Import block
import os
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.base import clone
import my_module_v2 as mx

pd.set_option('display.max_columns', None)

Section 2: Data Gathering and Cleaning

This section compiles the Moneypuck and Rotowire data into a format that is usable by the ML models.

There is some discrepencies between stats on ESPN and on moneypuck. These shouldn't alter the fantasy points too much. For example, Sydney Crosby is short-changed 1 faceoff win but has one additional hit in moneypuck than on ESPN, so in my model he has 0.10 more points than he does on ESPN. I would like to see why this is the case in the future/get fully accurate data (perhaps from ESPN themselves), but for now I am ok with this very small error.

In [3]:
yearly_player_data = []

for year in range(2010, current_year):
    moneypuck_data = mx.get_moneypuck_data(year)
    rotowire_data = mx.get_rotowire_data(year)
    combined_data = mx.combine_dataframes(moneypuck_data, rotowire_data)
    final_df = mx.calculate_fantasy_points(combined_data, points_dictionary)
    
    yearly_player_data.append(final_df)

    

This next cell compiles the yearly data into chunks of one, two, and three-year data to be used by the ML models.

In [4]:
ml_data_one_year = mx.get_ml_data(yearly_player_data, current_year, 1)
ml_data_two_years = mx.get_ml_data(yearly_player_data, current_year, 2)
ml_data_three_years = mx.get_ml_data(yearly_player_data, current_year, 3)

The data is now ready to be used to train the ML model.

In [5]:

one_year_X, one_year_y = mx.separate_fantasy_points(ml_data_one_year)
two_year_X, two_year_y = mx.separate_fantasy_points(ml_data_two_years)
three_year_X, three_year_y = mx.separate_fantasy_points(ml_data_three_years)

In [6]:
one_year_X.head()

,playerId,season,games_played,4on5_icetime,5on4_icetime,5on5_icetime,all_icetime,other_icetime,4on5_shifts,5on4_shifts,5on5_shifts,all_shifts,other_shifts,4on5_gameScore,5on4_gameScore,5on5_gameScore,all_gameScore,other_gameScore,4on5_onIce_xGoalsPercentage,5on4_onIce_xGoalsPercentage,5on5_onIce_xGoalsPercentage,all_onIce_xGoalsPercentage,other_onIce_xGoalsPercentage,4on5_offIce_xGoalsPercentage,5on4_offIce_xGoalsPercentage,5on5_offIce_xGoalsPercentage,all_offIce_xGoalsPercentage,other_offIce_xGoalsPercentage,4on5_onIce_corsiPercentage,5on4_onIce_corsiPercentage,5on5_onIce_corsiPercentage,all_onIce_corsiPercentage,other_onIce_corsiPercentage,4on5_offIce_corsiPercentage,5on4_offIce_corsiPercentage,5on5_offIce_corsiPercentage,all_offIce_corsiPercentage,other_offIce_corsiPercentage,4on5_onIce_fenwickPercentage,5on4_onIce_fenwickPercentage,5on5_onIce_fenwickPercentage,all_onIce_fenwickPercentage,other_onIce_fenwickPercentage,4on5_offIce_fenwickPercentage,5on4_offIce_fenwickPercentage,5on5_offIce_fenwickPercentage,all_offIce_fenwickPercentage,other_offIce_fenwickPercentage,4on5_iceTimeRank,5on4_iceTimeRank,5on5_iceTimeRank,all_iceTimeRank,other_iceTimeRank,4on5_I_F_xOnGoal,5on4_I_F_xOnGoal,5on5_I_F_xOnGoal,all_I_F_xOnGoal,other_I_F_xOnGoal,4on5_I_F_xGoals,5on4_I_F_xGoals,5on5_I_F_xGoals,all_I_F_xGoals,other_I_F_xGoals,4on5_I_F_xRebounds,5on4_I_F_xRebounds,5on5_I_F_xRebounds,all_I_F_xRebounds,other_I_F_xRebounds,4on5_I_F_xFreeze,5on4_I_F_xFreeze,5on5_I_F_xFreeze,all_I_F_xFreeze,other_I_F_xFreeze,4on5_I_F_xPlayStopped,5on4_I_F_xPlayStopped,5on5_I_F_xPlayStopped,all_I_F_xPlayStopped,other_I_F_xPlayStopped,4on5_I_F_xPlayContinuedInZone,5on4_I_F_xPlayContinuedInZone,5on5_I_F_xPlayContinuedInZone,all_I_F_xPlayContinuedInZone,other_I_F_xPlayContinuedInZone,4on5_I_F_xPlayContinuedOutsideZone,5on4_I_F_xPlayContinuedOutsideZone,5on5_I_F_xPlayContinuedOutsideZone,all_I_F_xPlayContinuedOutsideZone,other_I_F_xPlayContinuedOutsideZone,4on5_I_F_flurryAdjustedxGoals,5on4_I_F_flurryAdjustedxGoals,5on5_I_F_flurryAdjustedxGoals,all_I_F_flurryAdjustedxGoals,other_I_F_flurryAdjustedxGoals,4on5_I_F_scoreVenueAdjustedxGoals,5on4_I_F_scoreVenueAdjustedxGoals,5on5_I_F_scoreVenueAdjustedxGoals,all_I_F_scoreVenueAdjustedxGoals,other_I_F_scoreVenueAdjustedxGoals,4on5_I_F_flurryScoreVenueAdjustedxGoals,5on4_I_F_flurryScoreVenueAdjustedxGoals,5on5_I_F_flurryScoreVenueAdjustedxGoals,all_I_F_flurryScoreVenueAdjustedxGoals,other_I_F_flurryScoreVenueAdjustedxGoals,4on5_I_F_primaryAssists,5on4_I_F_primaryAssists,5on5_I_F_primaryAssists,all_I_F_primaryAssists,other_I_F_primaryAssists,4on5_I_F_secondaryAssists,5on4_I_F_secondaryAssists,5on5_I_F_secondaryAssists,all_I_F_secondaryAssists,other_I_F_secondaryAssists,4on5_I_F_shotsOnGoal,5on4_I_F_shotsOnGoal,5on5_I_F_shotsOnGoal,all_I_F_shotsOnGoal,other_I_F_shotsOnGoal,4on5_I_F_missedShots,5on4_I_F_missedShots,5on5_I_F_missedShots,all_I_F_missedShots,other_I_F_missedShots,4on5_I_F_blockedShotAttempts,5on4_I_F_blockedShotAttempts,5on5_I_F_blockedShotAttempts,all_I_F_blockedShotAttempts,other_I_F_blockedShotAttempts,4on5_I_F_shotAttempts,5on4_I_F_shotAttempts,5on5_I_F_shotAttempts,all_I_F_shotAttempts,other_I_F_shotAttempts,4on5_I_F_points,5on4_I_F_points,5on5_I_F_points,all_I_F_points,other_I_F_points,4on5_I_F_goals,5on4_I_F_goals,5on5_I_F_goals,Goals,other_I_F_goals,4on5_I_F_rebounds,5on4_I_F_rebounds,5on5_I_F_rebounds,all_I_F_rebounds,other_I_F_rebounds,4on5_I_F_reboundGoals,5on4_I_F_reboundGoals,5on5_I_F_reboundGoals,all_I_F_reboundGoals,other_I_F_reboundGoals,4on5_I_F_freeze,5on4_I_F_freeze,5on5_I_F_freeze,all_I_F_freeze,other_I_F_freeze,4on5_I_F_playStopped,5on4_I_F_playStopped,5on5_I_F_playStopped,all_I_F_playStopped,other_I_F_playStopped,4on5_I_F_playContinuedInZone,5on4_I_F_playContinuedInZone,5on5_I_F_playContinuedInZone,all_I_F_playContinuedInZone,other_I_F_playContinuedInZone,4on5_I_F_playContinuedOutsideZone,5on4_I_F_playContinuedOutsideZone,5on5_I_F_playContinuedOutsideZone,all_I_F_playContinuedOutsideZone,oth

In [ ]:
one_year_neural_nets = mx.create_models(
    one_year_X,
    one_year_y,
    MLPRegressor(max_iter=1000),
    number_of_one_year_neural_nets
)

In [ ]:
one_year_random_forests = mx.create_models(
    one_year_X,
    one_year_y,
    RandomForestRegressor(),
    number_of_one_year_random_forests
)

In [ ]:
two_year_neural_nets = mx.create_models(
    two_year_X,
    two_year_y,
    MLPRegressor(max_iter=1000),
    number_of_two_year_neural_nets
)

In [ ]:
two_year_random_forests = mx.create_models(
    two_year_X,
    two_year_y,
    RandomForestRegressor(),
    number_of_two_year_random_forests
)

In [ ]:
three_year_neural_nets = mx.create_models(
    three_year_X,
    three_year_y,
    MLPRegressor(max_iter=1000),
    number_of_three_year_neural_nets
)

In [ ]:
three_year_random_forests = mx.create_models(
    three_year_X,
    three_year_y,
    RandomForestRegressor(),
    number_of_three_year_random_forests
)

Now we generate the table with final predictions.